<a href="https://colab.research.google.com/github/seoil0728/CNN_Sentiment_Job_Analysis/blob/main/1D_CNN%EB%AA%A8%EB%8D%B8%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%A7%81%EC%97%85%EA%B0%90%EC%84%B1_%EB%B6%84%EC%84%9D_%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!mkdir data


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
mkdir: cannot create directory ‘data’: File exists


In [ ]:
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="./data/ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="./data/ratings_test.txt")

('./data/ratings_test.txt', <http.client.HTTPMessage at 0x7f9235a89090>)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dropout, Conv1D, GlobalMaxPooling1D, Dense, Input, Flatten, Concatenate
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import os
import pickle


def below_threshold_len(max_len, nested_list):
    count = 0
    for sentence in nested_list:
        if(len(sentence) <= max_len):
            count = count + 1
    print('전체 샘플 중 길이가 %s 이하인 샘플의 비율: %s'%(max_len, (count / len(nested_list))*100))


if not os.path.isfile('CNN_model.h5'):
    train_data = pd.read_table('./data/ratings_train.txt')
    test_data = pd.read_table('./data/ratings_test.txt')

    print('훈련용 리뷰 개수 :',len(train_data)) # 훈련용 리뷰 개수 출력
    train_data[:5] # 상위 5개 출력

    print('테스트용 리뷰 개수 :',len(test_data)) # 테스트용 리뷰 개수 출력
    test_data[:5]

    # document 열과 label 열의 중복을 제외한 값의 개수
    train_data['document'].nunique(), train_data['label'].nunique()

    # document 열의 중복 제거
    train_data.drop_duplicates(subset=['document'], inplace=True)

    print('총 샘플의 수 :',len(train_data))

    train_data['label'].value_counts().plot(kind = 'bar')

    print(train_data.groupby('label').size().reset_index(name = 'count'))

    print(train_data.isnull().values.any())

    print(train_data.isnull().sum())

    train_data.loc[train_data.document.isnull()]

    train_data = train_data.dropna(how = 'any') # Null 값이 존재하는 행 제거
    print(train_data.isnull().values.any()) # Null 값이 존재하는지 확인

    print(len(train_data))

    # 한글과 공백을 제외하고 모두 제거
    train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    train_data[:5]

    train_data['document'] = train_data['document'].str.replace('^ +', "") # white space 데이터를 empty value로 변경
    train_data['document'].replace('', np.nan, inplace=True)
    print(train_data.isnull().sum())

    train_data.loc[train_data.document.isnull()][:5]

    train_data = train_data.dropna(how = 'any')
    print(len(train_data))

    test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
    test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
    test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
    test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
    test_data = test_data.dropna(how='any') # Null 값 제거
    print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

    okt = Okt()
    okt.morphs('와 이런 것도 영화라고 차라리 뮤직비디오를 만드는 게 나을 뻔', stem = True)

    X_train = []
    for sentence in tqdm(train_data['document']):
        tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
        stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
        X_train.append(stopwords_removed_sentence)

    print(X_train[:3])

    X_test = []
    for sentence in tqdm(test_data['document']):
        tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
        stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
        X_test.append(stopwords_removed_sentence)


    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_train)

    print(tokenizer.word_index)

    threshold = 3
    total_cnt = len(tokenizer.word_index) # 단어의 수
    rare_cnt = 0 # 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
    total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
    rare_freq = 0 # 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합

    # 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
    for key, value in tokenizer.word_counts.items():
        total_freq = total_freq + value

        # 단어의 등장 빈도수가 threshold보다 작으면
        if(value < threshold):
            rare_cnt = rare_cnt + 1
            rare_freq = rare_freq + value

    print('단어 집합(vocabulary)의 크기 :',total_cnt)
    print('등장 빈도가 %s번 이하인 희귀 단어의 수: %s'%(threshold - 1, rare_cnt))
    print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
    print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

    # 전체 단어 개수 중 빈도수 2이하인 단어는 제거.
    # 0번 패딩 토큰을 고려하여 + 1
    vocab_size = total_cnt - rare_cnt + 1
    print('단어 집합의 크기 :',vocab_size)
    with open('vocab_size.pickle', 'wb') as f:
        pickle.dump(vocab_size, f, pickle.HIGHEST_PROTOCOL)
    with open('Xtrain.pickle', 'wb') as f:
        pickle.dump(X_train, f, pickle.HIGHEST_PROTOCOL)

    tokenizer = Tokenizer(vocab_size) 
    tokenizer.fit_on_texts(X_train)
    X_train = tokenizer.texts_to_sequences(X_train)
    X_test = tokenizer.texts_to_sequences(X_test)

    print(X_train[:3])

    y_train = np.array(train_data['label'])
    y_test = np.array(test_data['label'])

    drop_train = [index for index, sentence in enumerate(X_train) if len(sentence) < 1]

    # 빈 샘플들을 제거
    X_train = np.delete(X_train, drop_train, axis=0)
    y_train = np.delete(y_train, drop_train, axis=0)
    print(len(X_train))
    print(len(y_train))

    print('리뷰의 최대 길이 :',max(len(review) for review in X_train))
    print('리뷰의 평균 길이 :',sum(map(len, X_train))/len(X_train))
    plt.hist([len(review) for review in X_train], bins=50)
    plt.xlabel('length of samples')
    plt.ylabel('number of samples')
    plt.show()


    max_len = 30
    below_threshold_len(max_len, X_train)

    X_train = pad_sequences(X_train, maxlen=max_len)
    X_test = pad_sequences(X_test, maxlen=max_len)


    embedding_dim = 256
    dropout_ratio = (0.5, 0.8)
    num_filters = 128
    hidden_units = 256

    model_input = Input(shape = (max_len,))
    z = Embedding(vocab_size, embedding_dim, input_length = max_len, name="embedding")(model_input)
    z = Dropout(dropout_ratio[0])(z)

    conv_blocks = []

    for sz in [3, 4, 5]:
        conv = Conv1D(filters = num_filters,
                            kernel_size = sz,
                            padding = "valid",
                            activation = "relu",
                            strides = 1)(z)
        conv = GlobalMaxPooling1D()(conv)
        conv_blocks.append(conv)

    z = Concatenate()(conv_blocks) if len(conv_blocks) > 1 else conv_blocks[0]
    z = Dropout(dropout_ratio[1])(z)
    z = Dense(hidden_units, activation="relu")(z)
    model_output = Dense(1, activation="sigmoid")(z)

    model = Model(model_input, model_output)
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["acc"])

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    mc = ModelCheckpoint('CNN_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

    model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2, verbose=2, callbacks=[es, mc])
    with open('Xtest_data.pickle', 'wb') as f:
        pickle.dump(X_test, f, pickle.HIGHEST_PROTOCOL)

    with open('ytest_data.pickle', 'wb') as f:
        pickle.dump(y_test, f, pickle.HIGHEST_PROTOCOL)

loaded_model = load_model('CNN_model.h5')
with open('Xtest_data.pickle', 'rb') as f:
    X_test = pickle.load(f)

with open('ytest_data.pickle', 'rb') as f:
    y_test = pickle.load(f)

with open('vocab_size.pickle', 'rb') as f:
    vocab_size = pickle.load(f)

with open('Xtrain.pickle', 'rb') as f:
    X_train = pickle.load(f)

print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))



1527/1527 [==============================] - 5s 3ms/step - loss: 0.3614 - acc: 0.8426

 테스트 정확도: 0.8426


In [ ]:
def sentiment_predict(new_sentence):
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    okt = Okt()
    max_len = 30
    tokenizer = Tokenizer(vocab_size)
    tokenizer.fit_on_texts(X_train)
    new_sentence = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣 ]','', new_sentence)
    new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) # 정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = max_len) # 패딩
    score = float(loaded_model.predict(pad_new)) # 예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
        return 1
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))
        return 0


In [ ]:
def reading_get(filename):
    with open(filename, 'r') as f:
        data = f.read()

    fresh_data, older_data = data.split('(3, 4학년)')
    male_data = []
    female_data = []
    freshman = []
    olderman = []

    for i in fresh_data.split('\n'):
        if i != '':
            freshman.append(i[4:])

    for i in older_data.split('\n'):
        if i != '':
            olderman.append(i[4:])

    freshman.pop(0)

    for i in data.split('\n'):
        if i != '':
            if i[-1] == '남':
                male_data.append(i[4:])
            else:
                female_data.append(i[4:])

    female_data.remove('2학년)')
    female_data.remove('4학년)')

    return freshman, olderman, male_data, female_data

In [ ]:
print('공무원 한줄평 분석 결과')
filename = '/content/공무원 한줄평_221112_180111.txt'

freshes, olders, male, female = reading_get(filename)

score_of_freshes = []
score_of_olders = []
score_of_male = []
score_of_female = []

print('저학년')
for i in freshes:
    print(i)
    score_of_freshes.append(sentiment_predict(i))

print('저학년의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_freshes) / len(score_of_freshes)) * 100))
print()

print('고학년')
for i in olders:
    print(i)
    score_of_olders.append(sentiment_predict(i))

print('고학년의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_olders) / len(score_of_olders)) * 100))
print()

print('남자')
for i in male:
    print(i)
    score_of_male.append(sentiment_predict(i))

print('남자의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_male) / len(score_of_male)) * 100))
print()

print('여자')
for i in female:
    print(i)
    score_of_female.append(sentiment_predict(i))

print('여자의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_female) / len(score_of_female)) * 100))
print()

result = (sum(score_of_female) + sum(score_of_male)) / (len(score_of_female) + len(score_of_male))
print('공무원에 대한 전체 긍정 비율은 {:.2f}% 입니다.'.format(result * 100))

공무원 한줄평 분석 결과
저학년
나라가 망하는 것이 안니한 직업의 안정성이 높다 생각한다. 남
1/1 [==============================] - 0s 83ms/step
85.45% 확률로 부정 리뷰입니다.

수입이 낮은 직급은 낮지만 노후대비 관해서는 좋다고 생각한다. 남
1/1 [==============================] - 0s 16ms/step
68.61% 확률로 긍정 리뷰입니다.

9급은 잘 몰라도 7급은 꼭 하고 싶다. 남
1/1 [==============================] - 0s 14ms/step
80.94% 확률로 긍정 리뷰입니다.

안정적인 부분은 최고다. 하지만 너무 작은 사회라 너무 고였다. 남
1/1 [==============================] - 0s 15ms/step
93.57% 확률로 긍정 리뷰입니다.

수입을 제외한 모든 것을 가진 것 같다(9급기준), 현재 나의 목표이다. 남
1/1 [==============================] - 0s 17ms/step
68.06% 확률로 부정 리뷰입니다.

좋아요. 안정적인 수입 이거면 충분. 남
1/1 [==============================] - 0s 19ms/step
61.71% 확률로 긍정 리뷰입니다.

안정적인 직업인 것 같다. 여
1/1 [==============================] - 0s 21ms/step
72.35% 확률로 긍정 리뷰입니다.

하는 일이 똑가은 것 같아서 되고싶지 않다. 여
1/1 [==============================] - 0s 18ms/step
55.28% 확률로 부정 리뷰입니다.

지루할 것 같지만 별 생각 없다. 여
1/1 [==============================] - 0s 20ms/step
83.97% 확률로 부정 리뷰입니다.

업무가 지루할 것 같아서 별로 하고 싶지 않다. 여
1/1 [================

In [ ]:
print('산림조합 한줄평 분석 결과')
filename = '/content/산림조합 한줄평_221112_180223.txt'

freshes, olders, male, female = reading_get(filename)

score_of_freshes = []
score_of_olders = []
score_of_male = []
score_of_female = []

print('저학년')
for i in freshes:
    print(i)
    score_of_freshes.append(sentiment_predict(i))

print('저학년의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_freshes) / len(score_of_freshes)) * 100))
print()

print('고학년')
for i in olders:
    print(i)
    score_of_olders.append(sentiment_predict(i))

print('고학년의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_olders) / len(score_of_olders)) * 100))
print()

print('남자')
for i in male:
    print(i)
    score_of_male.append(sentiment_predict(i))

print('남자의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_male) / len(score_of_male)) * 100))
print()

print('여자')
for i in female:
    print(i)
    score_of_female.append(sentiment_predict(i))

print('여자의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_female) / len(score_of_female)) * 100))
print()

result = (sum(score_of_female) + sum(score_of_male)) / (len(score_of_female) + len(score_of_male))
print('산림조합에 대한 전체 긍정 비율은 {:.2f}% 입니다.'.format(result * 100))

산림조합 한줄평 분석 결과
저학년
수입이 높은 편에 속하긴 하지만 육체적으로 많이 힘들 것 같고, 이 때문에 선호하는 직업 중에서 가장 뒤떨어 진다고 생각한다. 여
1/1 [==============================] - 0s 15ms/step
87.28% 확률로 부정 리뷰입니다.

급여가 최고다. 하지만 그 뒤에 따르는 단점이 많다. 여
1/1 [==============================] - 0s 19ms/step
92.93% 확률로 긍정 리뷰입니다.

수입이라는 메리트가 있지만 일이 상당히 힘들 것 같다. 고려는 하고있다. 여
1/1 [==============================] - 0s 16ms/step
73.55% 확률로 부정 리뷰입니다.

so so. 몸은 힘들지만, 돈을 많이 줘서 좋아요. 여
1/1 [==============================] - 0s 15ms/step
63.12% 확률로 부정 리뷰입니다.

하는 일이 많은 것 같아서 되고 싶지 않다. 남
1/1 [==============================] - 0s 15ms/step
66.04% 확률로 긍정 리뷰입니다.

사회에 공헌할 수 있는 일이 많을 것 같아 일하게 되면 뿌듯할 것 같다. 남
1/1 [==============================] - 0s 16ms/step
81.16% 확률로 긍정 리뷰입니다.

돈은 많이 벌지만 매우 힘들 것 같아 하고 싶지 않다. 남
1/1 [==============================] - 0s 19ms/step
70.48% 확률로 부정 리뷰입니다.

생각 없다. 남
1/1 [==============================] - 0s 15ms/step
61.96% 확률로 부정 리뷰입니다.

활동적인 일이 많고 일이 고되다 들었다. 그만큼 돈을 더 받겠지만 힘들게 일하고 싶지 않다. 여
1/1 [==============================] - 0s 1

In [ ]:
print('연구원 한줄평 분석 결과')
filename = '/content/연구원 한줄평_221112_180311.txt'

freshes, olders, male, female = reading_get(filename)

score_of_freshes = []
score_of_olders = []
score_of_male = []
score_of_female = []

print('저학년')
for i in freshes:
    print(i)
    score_of_freshes.append(sentiment_predict(i))

print('저학년의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_freshes) / len(score_of_freshes)) * 100))
print()

print('고학년')
for i in olders:
    print(i)
    score_of_olders.append(sentiment_predict(i))

print('고학년의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_olders) / len(score_of_olders)) * 100))
print()

print('남자')
for i in male:
    print(i)
    score_of_male.append(sentiment_predict(i))

print('남자의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_male) / len(score_of_male)) * 100))
print()

print('여자')
for i in female:
    print(i)
    score_of_female.append(sentiment_predict(i))

print('여자의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_female) / len(score_of_female)) * 100))
print()

result = (sum(score_of_female) + sum(score_of_male)) / (len(score_of_female) + len(score_of_male))
print('연구원에 대한 전체 긍정 비율은 {:.2f}% 입니다.'.format(result * 100))

연구원 한줄평 분석 결과
저학년
밤을 새는 일도 많을 것 같아 생활패턴이 불규칙 해져 되고싶지 않다. 남
1/1 [==============================] - 0s 18ms/step
61.27% 확률로 부정 리뷰입니다.

하지만 전문성이 강하다는 생각이 들어 멋지다고 생각한다. 남
1/1 [==============================] - 0s 16ms/step
84.00% 확률로 긍정 리뷰입니다.

전문성을 많이 요구하여 공부를 많이 해야하는 등 되기 어렵지만 한 분야에 대한 전문가로서 뿌듯함은 가질 수 있을 것 같다. 남
1/1 [==============================] - 0s 16ms/step
81.15% 확률로 긍정 리뷰입니다.

이건 좀... 개인적으로 좋은데 수익이 안 좋을 것 같다. 남
1/1 [==============================] - 0s 15ms/step
62.92% 확률로 긍정 리뷰입니다.

공무원과 산림조합의 밸런스가 섞여있다. 혼자 진득하게 연구한다면 좋겠지만 나는 그렇게 못 한다. 남
1/1 [==============================] - 0s 15ms/step
60.97% 확률로 부정 리뷰입니다.

엥간한 사명감 아니면 하지 못한다. 여
1/1 [==============================] - 0s 15ms/step
66.13% 확률로 부정 리뷰입니다.

대학의 이미지가 워낙 안 좋기 때문에 연구원은 가장 비선호에 가깝다. 여
1/1 [==============================] - 0s 17ms/step
71.03% 확률로 부정 리뷰입니다.

기피 하는 이유로는 근무시간이 가장 많을 것 같고 이에 정신적, 육체적 피로도 가장 많을 것 같다. 여
1/1 [==============================] - 0s 15ms/step
60.88% 확률로 부정 리뷰입니다.

연구를 계속 하는 건 재미가 없을 것 같아 되고 싶지 않다

In [ ]:
print('임업진흥원 한줄평 분석 결과')
filename = '/content/임업진흥원 한줄평_221112_180320.txt'

freshes, olders, male, female = reading_get(filename)

score_of_freshes = []
score_of_olders = []
score_of_male = []
score_of_female = []

print('저학년')
for i in freshes:
    print(i)
    score_of_freshes.append(sentiment_predict(i))

print('저학년의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_freshes) / len(score_of_freshes)) * 100))
print()

print('고학년')
for i in olders:
    print(i)
    score_of_olders.append(sentiment_predict(i))

print('고학년의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_olders) / len(score_of_olders)) * 100))
print()

print('남자')
for i in male:
    print(i)
    score_of_male.append(sentiment_predict(i))

print('남자의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_male) / len(score_of_male)) * 100))
print()

print('여자')
for i in female:
    print(i)
    score_of_female.append(sentiment_predict(i))

print('여자의 평균 긍정 확률은 {:.2f}% 입니다.'.format((sum(score_of_female) / len(score_of_female)) * 100))
print()

result = (sum(score_of_female) + sum(score_of_male)) / (len(score_of_female) + len(score_of_male))
print('임업진흥원에 대한 전체 긍정 비율은 {:.2f}% 입니다.'.format(result * 100))

임업진흥원 한줄평 분석 결과
저학년
하는 일이 다양할 것 같고 수입도 어느 정도 괜찮을 것 같아 공기업에 취업하게 되면 좋을 것 같다. 남
1/1 [==============================] - 0s 22ms/step
57.35% 확률로 긍정 리뷰입니다.

되기 어렵지만 근무환경, 조건이 좋을 것 같다. 남
1/1 [==============================] - 0s 16ms/step
72.43% 확률로 긍정 리뷰입니다.

좋아요. 공무원보다는 아니어도 안정적이고 수입이 so so 하다. 남
1/1 [==============================] - 0s 21ms/step
51.14% 확률로 부정 리뷰입니다.

모두가 가장 원하고 나도 원했지만 하기 힘들다(수많은 공부량). 남
1/1 [==============================] - 0s 15ms/step
58.00% 확률로 긍정 리뷰입니다.

급여, 워라벨 최상이라 생각한다. 하지만 되는 것은 매우 어렵다. 여
1/1 [==============================] - 0s 14ms/step
60.38% 확률로 부정 리뷰입니다.

일단 공기업에 대한 주위의 인식이 좋고 수입도 높은 편이라 모두가 선호하는 직업일 것 같다. 여
1/1 [==============================] - 0s 18ms/step
74.49% 확률로 부정 리뷰입니다.

다른 직업들 중에서 가장 되고 싶은 직업이기도 하고 선호하는 가장 큰 이유는 수입이다. 여
1/1 [==============================] - 0s 15ms/step
52.98% 확률로 부정 리뷰입니다.

그리고 수입 대비 업무환경도 좋은 것 같다는 인식도 선호하는 이유 중 하나이다. 여
1/1 [==============================] - 0s 18ms/step
67.18% 확률로 긍정 리뷰입니다.

생각없다. 여
1/1 [====================